# Capstone Project Notes

**Competition Page:** https://www.kaggle.com/competitions/agents-intensive-capstone-project/overview 

**Notes:** https://docs.google.com/document/d/1Hs3YYGIv72_GXaMWJkluOxXD7sn3cxz2kVfSPHEJOwQ/edit?tab=t.krqovzz0jw2x

**Outline:**
- Chat interface (Multi agent system)
- Agent to search Pubmed and rank results (Custom tools)
- Agent to summarize papers? (Custom tools)
- Agent for insurance data? (Custom tools)
- Agent to aggregate results (Multi agent system)

In [ ]:
import logging
import os
from dotenv import load_dotenv


# Load environment variables from the .env file (if present)
load_dotenv()

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
try:
    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    print("✅ API key setup complete.")
except KeyError:
    raise KeyError("GOOGLE_API_KEY not found in environment variables.")

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

In [ ]:
# Only need to run this cell once I think

# !mkdir researchagent

# !rm -rf researchagent

# !adk create researchagent --model gemini-2.5-flash-lite --api_key $GEMINI_API_KEY

In [ ]:
# %%writefile researchagent/agent.py

from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService

from google.adk.runners import Runner
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.genai import types


APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session
MODEL_NAME = "gemini-2.5-flash-lite"

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

# MCP integration
mcp_pubmed_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@cyanheads/pubmed-mcp-server",
            ],
            env={
                "NBCBI_API_KEY": os.getenv("NBCBI_API_KEY", "")
            }
        ),
        timeout=30,

    )
)

# Pubmed Search Agent
pubmed_search_agent = LlmAgent(
    name="pubmed_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Pubmed",
    instruction="""You are a trained professional researcher. Use 'pubmed_search_articles' in the mcp_pubmed_server tool to find information on the given topic. Return a total of 10 artcles. The articles should be a mixture of review articles and primary research articles. You must also get URL for the article or the PMID Focus on recent articles from the last 5 years unless the topic requires foundational knowledge from older articles. Focus on high-impact journals and reputable sources.""",
    tools=[mcp_pubmed_server],
    # output_key="pmids"
)

# Summarize Agent
summarize_agent = LlmAgent(
    name="summarize_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Summarizes research papers",
    instruction="""You are a research professional working in the area of science communication for the general public. Summarise the research papers you are given by getting the full text of the papers based on the PMIDs in {pmids}. Construct a report that summarizes the findings from the abstracts. The document should be understandable for non scietifically trained citizen. Please be sure to define any acronyms. Please include citations of the abstracts (no making anything up!). Aim to keep the report under 500 words. If the article is not accessible, provide a summary based on the abstract only. If no abstract is available, state that the article could not be accessed.""",
    tools=[mcp_pubmed_server],
    # output_key="summaries"
)


root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and summarize them. 
    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'pubmed_search_agent'. 
    2) Then, pass the papers to 'summarize_agent' tool to summarize the papers.
    3) Check that summaries include a citation with a link to the article. If not, go back to step 2 and ask the summarize_agent to include citations with links.
    4) Finally, provide a comprehensive response to the user that includes the summaries and citations
    Make sure to ALWAYS use the tools provided to you. NEVER make up information or citations.
    """,
    # sub_agents=[pubmed_search_agent, summarize_agent]
    tools=[AgentTool(agent=pubmed_search_agent), AgentTool(agent=summarize_agent)]
)



In [ ]:
# async def run_session(
#     runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
# ):
#     """Helper function to run queries in a session and display responses."""
#     print(f"\n### Session: {session_id}")

#     # Create or retrieve session
#     try:
#         session = await session_service.create_session(
#             app_name=APP_NAME, user_id=USER_ID, session_id=session_id
#         )
#     except:
#         session = await session_service.get_session(
#             app_name=APP_NAME, user_id=USER_ID, session_id=session_id
#         )

#     # Convert single query to list
#     if isinstance(user_queries, str):
#         user_queries = [user_queries]

#     # Process each query
#     for query in user_queries:
#         print(f"\nUser > {query}")
#         query_content = types.Content(role="user", parts=[types.Part(text=query)])

#         # Stream agent response
#         async for event in runner_instance.run_async(
#             user_id=USER_ID, session_id=session.id, new_message=query_content
#         ):
#             if event.is_final_response() and event.content and event.content.parts:
#                 text = event.content.parts[0].text
#                 if text and text != "None":
#                     print(f"Model: > {text}")


In [ ]:
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)
from google.adk.plugins.reflect_retry_tool_plugin import (
    ReflectAndRetryToolPlugin,
)

# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner with persistent storage
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME, 
    session_service=session_service,
    plugins=[
        LoggingPlugin(), # Add the plugin. Handles standard Observability logging across ALL agents
        ReflectAndRetryToolPlugin(),
    ],
)

In [ ]:
QUERY = "How does glycemic control change with increased in testosterone levels?"
# QUERY = "How does hormone replacement therapy, including both testosterone and estrogen, effect your A1C?"
# QUERY = "What are the main causes of acute respiratory distress syndrome?"
# QUERY = "I want to learn about stage 4 prostate cancer. What can cause it?"
# QUERY = "Yes. What's the best treatment for stage 4 prostate cancer?"
# QUERY = "Can you tell me more about genomically targeted therapies"

# Option 1
response = await runner.run_debug(QUERY, verbose=True)

In [ ]:
print (response)

# Observability Code

In [ ]:
# View session events

# session = await session_service.get_session(
#     app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
# )

# # Let's see what's in the session
# print("📝 Session contains:")
# for event in session.events:
#     # print(event) # Uncomment this line to view the full event
#     text = (
#         event.content.parts[0].text[:6000]
#         if event.content and event.content.parts and event.content.parts[0].text
#         else "(empty)"
#     )
#     print(f"  {event.content.role}: {text}")

In [ ]:
# get_adk_proxy_url

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


url_prefix = get_adk_proxy_url()

In [ ]:
# TODO: this doesn't work but idk if we need a log file

# Check the DEBUG logs from the broken agent
# print("🔍 Examining web server logs for debugging clues...\n")
# !cat logger.log

# 